In [ ]:
from QLBM import QLBM, collision, InitializeQC
import numpy as np
import matplotlib.pyplot as plt
import qiskit_aer
from qiskit import transpile

In [ ]:
# Domain and grid setup
N_POINTS_X, N_POINTS_Y = 32, 16
x_0 = np.arange(N_POINTS_X)
y_0 = np.arange(N_POINTS_Y)
X, Y = np.meshgrid(x_0, y_0)  # Create a 2D grid

# Simulation parameters
TIMESTEPS = 50
NUMBER_DISCRETE_VELOCITIES = 9  # D2Q9 lattice configuration

# Velocity field initialization
u = np.full((N_POINTS_X, N_POINTS_Y), 0.2)      # Initialize u component
u[:, int(N_POINTS_Y / 2):] = -0.2               # Set negative u component in the upper half
v = np.full((N_POINTS_X, N_POINTS_Y), 0.1)      # Initialize v component

# Scalar field initialization
Psi_init = np.full((N_POINTS_X, N_POINTS_Y), 0.1)

# Cylinder parameters
CYLINDER_CENTER_INDEX_X = N_POINTS_X / 2
CYLINDER_CENTER_INDEX_Y = N_POINTS_Y / 2
CYLINDER_RADIUS_OUTER = 4
CYLINDER_RADIUS_INNER = 2

# Create a ring shape within the scalar field
distance_from_center = np.sqrt((X - CYLINDER_CENTER_INDEX_X)**2 + (Y - CYLINDER_CENTER_INDEX_Y)**2)
ring = ((distance_from_center < CYLINDER_RADIUS_OUTER) & 
        (distance_from_center > CYLINDER_RADIUS_INNER)).T
Psi_init[ring] = 0.4  # Set higher concentration in the ring region

In [ ]:
simulator = qiskit_aer.backends.statevector_simulator.StatevectorSimulator()

In [ ]:
# Initialize the quantum LBM scalar field
Psi_qlbm = np.zeros((TIMESTEPS + 1, N_POINTS_X, N_POINTS_Y))
Psi_qlbm[0, :, :] = Psi_init

# Quantum LBM simulation loop
for t in range(TIMESTEPS):
    # Initialize velocity field for the current timestep
    u_LBM = np.ones((N_POINTS_X, N_POINTS_Y, 2))
    u_LBM[:, :, 0] = u  # Set the x-component of the velocity
    u_LBM[:, :, 1] = v  # Set the y-component of the velocity

    # Create and run the quantum circuit for LBM
    qc = QLBM(density_field=Psi_qlbm[t, :, :], velocity_field=u_LBM, number_velocities=NUMBER_DISCRETE_VELOCITIES)
    compiled_circuit = transpile(qc, simulator)
    result = simulator.run(compiled_circuit).result()
    
    # Process the quantum statevector to update Psi_qlbm
    statevector = np.array(result.get_statevector())
    real_part_statevector = np.real(statevector[:N_POINTS_X * N_POINTS_Y])
    real_part_statevector_reshaped = np.reshape(real_part_statevector, (N_POINTS_X, N_POINTS_Y), order='F')
    
    # Normalize and update the scalar field for the next timestep
    Psi_qlbm[t + 1, :, :] = real_part_statevector_reshaped * np.linalg.norm(Psi_qlbm[t, :, :].flatten()) * 4

In [ ]:
plt.rcParams.update({'font.size': 35})
plt.rcParams['text.usetex'] = True
major_ticks = np.arange(0, 33, 5)
minor_ticks = np.arange(0, 33, 5)
timestep=-1
fig, ax = plt.subplots(1, figsize=(18.5, 10.5), dpi=100)
ax.set_xticks(major_ticks)
ax.set_yticks(major_ticks)
plt.imshow(Psi_qlbm[timestep,:,:].T)
ax.invert_yaxis()
ax.grid()
ax.set_xlabel('X')
ax.set_ylabel('Y')

In [ ]:
# Lattice parameters for D2Q9
LATTICE_VELOCITIES = np.array([
    [ 0,  1,  0, -1,  0,  1, -1, -1,  1],
    [ 0,  0,  1,  0, -1,  1,  1, -1, -1]
])

LATTICE_WEIGHTS = np.array([
    4/9,                        # Center Velocity
    1/9, 1/9, 1/9, 1/9,         # Axis-Aligned Velocities
    1/36, 1/36, 1/36, 1/36      # Diagonal Velocities
])

# Initialize the scalar field for classical LBM
Psi_classical = np.zeros((TIMESTEPS + 1, N_POINTS_X, N_POINTS_Y))
Psi_classical[0, :, :] = Psi_init

# Speed of sound
c_s = 1 / np.sqrt(3)

# Classical LBM loop
for t in range(TIMESTEPS):
    # Calculate equilibrium distribution functions for each velocity direction
    f0 = LATTICE_WEIGHTS[0] * Psi_classical[t, :, :]
    
    f = [f0]  # List to hold all f components including f0 for easy summing later
    for i in range(1, 9):
        velocity_projection = (LATTICE_VELOCITIES[0, i] * u + LATTICE_VELOCITIES[1, i] * v) / (c_s**2)
        fi = LATTICE_WEIGHTS[i] * Psi_classical[t, :, :] * (1 + velocity_projection)
        f.append(fi)

    # Streaming step: Shift each distribution along its corresponding velocity direction
    for i in range(1, 9):
        f[i] = np.roll(np.roll(f[i], LATTICE_VELOCITIES[1, i], axis=1), LATTICE_VELOCITIES[0, i], axis=0)

    # Update scalar field by summing over all distributions
    Psi_classical[t + 1, :, :] = sum(f)

In [ ]:
# Initialize MSE and RMSE arrays
MSE_opt = np.zeros(TIMESTEPS)
RMSE_opt = np.zeros(TIMESTEPS)

# Remove initial state from classical and quantum fields
Psi_classical_no_init = Psi_classical[1:, :, :]
Psi_quantum_no_init = Psi_qlbm[1:, :, :]

# Calculate MSE and RMSE for each timestep
for t in range(TIMESTEPS):
    difference = Psi_quantum_no_init[t, :, :] - Psi_classical_no_init[t, :, :]
    MSE_opt[t] = np.mean(np.square(difference))
    RMSE_opt[t] = np.sqrt(MSE_opt[t])

In [ ]:
plt.rcParams.update({'font.size': 35})
plt.rcParams['text.usetex'] = True
plt.figure(1,figsize=(18.5, 10.5), dpi=100)
plt.plot(list(range(1, TIMESTEPS+1)),RMSE_opt, linewidth=3)
plt.grid()
plt.legend(['D2Q9'],loc=0)
plt.title('RMSE between digital and quantum LBM')
plt.ylabel('error')
plt.xlabel('time step')
plt.draw()